# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\amizrahi\source\repos\TextAnalysis


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [10]:
# Method that remove all the unnecessary chars. Like: double space or quotation marks or special chars
def remove_unnecessary_chars(series):
    x = list(series)
    preprocess_x = []

    for txt in range(0, len(x)):
      txt = re.sub(r'\W', ' ', str(x[txt]))
      txt = re.sub(r'\s+[a-zA-Z]\s+', ' ', txt)
      txt = re.sub(r'\^[a-zA-Z]\s+', ' ', txt)
      txt = re.sub(r'\s+', ' ', txt, flags=re.I)
      txt = re.sub(r'^b\s+', '', txt)
      txt = txt.lower()
      preprocess_x.append(txt)

    return preprocess_x

In [11]:
# Method that make sure that our story contain only hebrew words
def hebrew_text(story):
    list_of_the_tokens = []
    all_the_tokens = ht.tokenize(story)

    for grp, token, token_num, (start_index, end_index) in all_the_tokens:
        if grp == 'HEBREW':
            list_of_the_tokens.append(token)
    return list_of_the_tokens

In [23]:
# Remove unnecessary chars from the stories by using regex
train_X = remove_unnecessary_chars(df_train.story)
test_X = remove_unnecessary_chars(df_test.story)

In [24]:
# Coverts gender from [m, f] to [0,1] to ignore errors when I fit the y train
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(df_train.gender)

In [25]:
# Transform the training data using count vectorizer
count_vect = CountVectorizer(analyzer='word', tokenizer=hebrew_text)
count_vect.fit(train_X)
train_X = count_vect.transform(train_X)
test_X = count_vect.transform(test_X)
train_X = preprocessing.normalize(train_X, norm='l1')

CountVectorizer(tokenizer=<function hebrew_text at 0x000002878D1F2290>)

In [26]:
# I used decision tree Classifier
classifier = DecisionTreeClassifier(max_depth=15)
classifier.fit(train_X, train_y)

DecisionTreeClassifier(max_depth=15)

In [27]:
# Train the data
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.2, random_state=0)

In [28]:
# Predict the tests and print the accuracy
valid_pred = classifier.predict(valid_X)
print('Validation Accuracy:', metrics.accuracy_score(valid_pred, valid_y))

Validation Accuracy: 0.9470198675496688


In [29]:
# Get the f1 score normal and average. The first solution is to run the equation and the second solution is by method
f1 = f1_score(valid_pred, valid_y)
f1_female, f1_male = f1_score(valid_pred, valid_y, average=None)
f1_macro = f1_score(valid_pred, valid_y, average='macro')
print('F1-score: ', f1)
print('Female f1-score: ', f1_female)
print('Male f1-score: ', f1_male)
print('F1-score average equation: ', (f1_male + f1_female) / 2)
print('F1-score average method: ', f1_macro)

F1-score:  0.9661016949152543
Female f1-score:  0.8787878787878788
Male f1-score:  0.9661016949152543
F1-score average equation:  0.9224447868515666
F1-score average method:  0.9224447868515666


In [31]:
# Create the dataframe of the prediction
y_predict = classifier.predict(test_X)
df_predicted = df_test.copy()
df_predicted.story = np.where(y_predict == 0, 'm', 'f')
df_predicted.rename(columns={'story': 'df_predicted'}, inplace=True)

In [32]:
df_predicted.head()

,test_example_id,df_predicted
0,0,f
1,1,m
2,2,m
3,3,f
4,4,f


In [33]:
df_predicted.tail()

,test_example_id,df_predicted
318,318,f
319,319,f
320,320,m
321,321,m
322,322,f


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [34]:
df_predicted.to_csv('classification_results.csv',index=False)